In [22]:
import geopandas as gpd
import numpy as np
from shapely.ops import cascaded_union
from shapely.geometry import Polygon

In [4]:
predicted_path = "/home/user/projects/clearcut_detection/data/predicted/predicted_20160103_66979721-be1b-4451-84e0-4a573236defd.geojson"
pred = gpd.read_file(predicted_path)

In [21]:
sum_coords = []
for num, row in pred.iterrows():
    sum_coords.extend(list(row['geometry'].exterior.coords))

In [108]:
gt_poly.crs = {'init' :'epsg:4326'}

In [111]:
gt_poly = gt_poly.to_crs({'init': "epsg:32637"})

In [112]:
sum_coords = []
for num, row in gt_poly.iterrows():
    sum_coords.extend(list(row['geometry'].exterior.coords))

In [113]:
base_bound = Polygon(sum_coords).bounds

In [114]:
p2 = Polygon(
            [(base_bound[0], base_bound[1]),
             (base_bound[0], base_bound[3]),
             (base_bound[2], base_bound[3]),
             (base_bound[2], base_bound[1])
             ])

In [115]:
p2.exterior.coords.xy

(array('d', [262575.75868537056, 262575.75868537056, 356329.76694924606, 356329.76694924606, 262575.75868537056]),
 array('d', [5478583.44112516, 5581428.357466508, 5581428.357466508, 5478583.44112516, 5478583.44112516]))

In [120]:
p3 = p1.intersection(p2)

In [56]:
p1 = Polygon(
            [(base_bound[0], base_bound[1]),
             (base_bound[0], base_bound[3]),
             (base_bound[2], base_bound[3]),
             (base_bound[2], base_bound[1])
             ])

In [121]:
united_poly = gpd.GeoDataFrame(geometry=[p3])
united_poly.to_file("/home/user/projects/clearcut_detection/data/predicted/intersection_f1.geojson", driver="GeoJSON")

In [124]:
gt_for_f1 = gt_poly[gt_poly.intersects(p3)]

In [125]:
pred_for_f1 = pred[pred.intersects(p3)]

In [126]:
pred_for_f1.to_file("/home/user/projects/clearcut_detection/data/predicted/res_for_f1.geojson", driver="GeoJSON")

In [127]:
gt_for_f1.to_file("/home/user/projects/clearcut_detection/data/predicted/gt_for_f1.geojson", driver="GeoJSON")

In [68]:
import fiona
from shapely.geometry import shape

In [90]:
shp_collection = fiona.open(gt_path)
# shp_geom = []
for i in shp_collection:
    i['geometry']
#     shp_geom.append(crds)

ValueError: year is out of range

In [92]:
gt_path = "/home/user/projects/clearcut_detection/data/s2_data/markup/clearcuts_backup_2016-08-30/clearcuts_backup_2016-08-30.shp"
gt_poly = gpd.read_file(gt_path)

In [128]:
def F1_score(pred_polys, true_polys, threshold):
    """
    Calculates F1 score using shapely polygons.

    :param pred_polys: Predicted polygons.
    :param true_polys: Ground truth polygons.
    :param threshold: IoU threshold.
    :return: F1 score, recall, precision.
    """
    preds = [pred.buffer(0) for pred in pred_polys]
    truths = [true.buffer(0) for true in true_polys]

    if (len(preds) == 0) and (len(truths) == 0):
        precision = 1
        recall = 1
        F1 = 1
        return F1, recall, precision

    true_positives = 0
    false_positives = 0
    false_negatives = 0

    for truth in truths:
        ious = []
        for pred in preds:
            if not truth.intersects(pred):
                ious.append(0)
            else:
                intersection = truth.intersection(pred).area
                union = truth.union(pred).area
                ious.append(intersection / union)

        if len(ious) == 0:
            false_negatives += 1
            continue

        best = max(ious)
        if best >= threshold:
            preds.pop(ious.index(best))
            true_positives += 1
        else:
            false_negatives += 1

    false_positives = len(preds)

    try:
        precision = true_positives / (true_positives + false_positives)
    except:
        print("Precision value undefined, setting to zero")
        precision = 0

    try:
        recall = true_positives / (true_positives + false_negatives)
    except:
        print("Recall value undefined, setting to zero")
        recall = 0

    try:
        F1 = (
            2 * (precision * recall) / (precision + recall)
        )
    except:
        print("F1 value undefined, setting to zero")
        F1 = 0
    return F1, recall, precision


In [138]:
areas = [_poly.area for _poly in gt_for_f1['geometry'].values]

In [149]:
np.max(areas)

610982.4873263139

In [162]:
pred_areas = [_poly.area for _poly in pred_for_f1['geometry'].values]

In [176]:
np.max(pred_areas)

789500.0

In [157]:
len(pred_filtered)

721

In [158]:
len(gt_for_f1['geometry'].values)

970

In [164]:
pred_filtered = pred_for_f1['geometry'].values[np.array(pred_areas) > 2950]

In [173]:
metrics = F1_score(pred_filtered, gt_for_f1['geometry'].values, 0.5)
F1 = metrics[0]

In [175]:
metrics

(0.4665482534043813, 0.4061855670103093, 0.5479833101529903)